In [ ]:
#iou 1 # trying attention unet
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from tqdm import tqdm
from skimage.io import imread
from skimage import img_as_float
from scipy import signal
from numpy import pad
from sklearn.metrics import precision_recall_fscore_support
from torchvision import transforms
from scipy.ndimage import binary_closing

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

drive.mount('/content/drive')

# ### Feature: Explore files - List contents of Google Drive to understand dataset location (Screenshot 1)
print("\nContents of /content/drive/MyDrive:")
try:
    print(os.listdir("/content/drive/MyDrive"))
except Exception as e:
    print(f"Error listing drive contents: {e}")

# ### Feature: Understand the data - Prompt user for dataset path to ensure correct data is used (Screenshot 1)
correct_path_found = False
while not correct_path_found:
    dataset_path = input("\nPlease enter the full path to your dataset folder (or 'quit' to exit): ")
    if dataset_path.lower() == 'quit':
        raise FileNotFoundError("Please manually check your Google Drive and update the path in the code.")
    if os.path.exists(dataset_path):
        print(f"\nFound dataset at: {dataset_path}")
        print(f"Contents of dataset folder: {os.listdir(dataset_path)}")
        correct_path_found = True
    else:
        print(f"\nPath {dataset_path} not found. Please try again.")

image_dirs = [
    os.path.join(dataset_path, 'benign'),
    os.path.join(dataset_path, 'malignant'),
    os.path.join(dataset_path, 'normal')
]

# ### Feature: Explore files - Inspect folder structure, image formats, and annotation files (Screenshot 1)
for directory in image_dirs:
    if os.path.exists(directory):
        print(f"Found directory: {directory}")
        img_count = len([f for f in os.listdir(directory) if f.endswith('.png') and '_mask' not in f])
        mask_count = len([f for f in os.listdir(directory) if f.endswith('_mask.png')])
        print(f"  Contains {img_count} images and {mask_count} masks")
    else:
        print(f"Warning: Directory {directory} not found")

# ### Feature: Preprocess images - Noise reduction using Gaussian filter (Screenshot 1)
def gaussian_kernel(width=1, size=5):
    x = np.linspace(-size // 2, size // 2, size)
    gaussian1d = np.exp(-0.5 * (x / width) ** 2)
    gaussian1d /= np.sum(gaussian1d)
    kernel = np.outer(gaussian1d, gaussian1d)
    return kernel / np.sum(kernel)

def apply_filter(image, kernel):
    pad_width = ((kernel.shape[0] // 2, kernel.shape[0] // 2),
                 (kernel.shape[1] // 2, kernel.shape[1] // 2))
    padded_img = pad(image, pad_width, mode='reflect')
    filtered = signal.convolve2d(padded_img, kernel, mode='valid', boundary='symm')
    return filtered

# ### Feature: Preprocess images - Resize images to 256x256 (Screenshot 1)
def resize_image(image, target_size=(256, 256)):
    h, w = image.shape
    new_h, new_w = target_size
    i_coords = np.linspace(0, h - 1, new_h, dtype=np.float32)
    j_coords = np.linspace(0, w - 1, new_w, dtype=np.float32)
    i_indices = np.round(i_coords).astype(np.int32)
    j_indices = np.round(j_coords).astype(np.int32)
    i_indices = np.clip(i_indices, 0, h - 1)
    j_indices = np.clip(j_indices, 0, w - 1)
    resized = image[i_indices][:, j_indices]
    return resized

# ### Feature: Preprocess images - Normalize pixel values (Screenshot 1)
def normalize_image(image):
    return image / image.max() if image.max() != 0 else image

# ### Feature: Parse and verify lesion masks and image-level labels with augmentation (Screenshot 1)
class BreastUltrasoundDataset(Dataset):
    def __init__(self, image_dirs, transform=None):
        self.transform = transform or transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
        ])
        self.samples = []
        for label_idx, label_dir in enumerate(image_dirs):
            if not os.path.exists(label_dir):
                print(f"Warning: Directory {label_dir} not found")
                continue
            try:
                images = [f for f in sorted(os.listdir(label_dir))
                         if f.endswith('.png') and '_mask' not in f]
                for img_name in images:
                    img_path = os.path.join(label_dir, img_name)
                    mask_path = os.path.join(label_dir, img_name.replace('.png', '_mask.png'))
                    if os.path.exists(mask_path):
                        mask = img_as_float(imread(mask_path))
                        if mask.ndim == 3:
                            mask = img_as_float(np.mean(mask, axis=2))
                        if np.max(mask) > 0:
                            self.samples.append((img_path, mask_path, label_idx))
                        else:
                            print(f"Skipping sample with empty mask: {mask_path}")
                    else:
                        print(f"Warning: Missing mask for {img_path}")
            except Exception as e:
                print(f"Error processing {label_dir}: {str(e)}")
        if not self.samples:
            raise RuntimeError("No valid image-mask pairs found in dataset directories")
        print(f"\nDataset successfully initialized with {len(self.samples)} valid samples (BUSI dataset with speckle noise)")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, mask_path, label = self.samples[idx]
        try:
            img = img_as_float(imread(img_path))
            if img.ndim == 3:
                img = img_as_float(np.mean(img, axis=2))
            mask = img_as_float(imread(mask_path))
            if mask.ndim == 3:
                mask = img_as_float(np.mean(mask, axis=2))
            gaussian_kern = gaussian_kernel(width=1, size=5)
            img = apply_filter(img, gaussian_kern)
            img = resize_image(img, target_size=(256, 256))
            mask = resize_image(mask, target_size=(256, 256))
            img = normalize_image(img)
            mask = (mask > 0.5).astype(np.float32)

            # Apply augmentation
            if self.transform:
                img_tensor = torch.tensor(img[np.newaxis, :, :], dtype=torch.float32)
                mask_tensor = torch.tensor(mask[np.newaxis, :, :], dtype=torch.float32)
                augmented = self.transform({"image": img_tensor, "mask": mask_tensor})
                img = augmented["image"].squeeze(0).numpy()
                mask = augmented["mask"].squeeze(0).numpy()

            img_rgb = np.stack([img, img, img], axis=-1)
            mask = np.expand_dims(mask, axis=-1)
            sample = {
                'image': torch.tensor(img_rgb, dtype=torch.float32).permute(2, 0, 1),
                'mask': torch.tensor(mask, dtype=torch.float32).permute(2, 0, 1),
                'label': torch.tensor(label, dtype=torch.long)
            }
            return sample
        except Exception as e:
            print(f"Error loading {img_path}: {str(e)}")
            return {
                'image': torch.zeros((3, 256, 256), dtype=torch.float32),
                'mask': torch.zeros((1, 256, 256), dtype=torch.float32),
                'label': torch.tensor(label, dtype=torch.long)
            }

# ### Loss Function: Weighted Dice + BCE Loss with class weighting
def dice_bce_loss(pred, target, smooth=1e-5, weight=0.8):
    pred = torch.sigmoid(pred)
    bce = F.binary_cross_entropy(pred, target)
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    dice = 1 - (2. * intersection + smooth) / (union + smooth)
    # Class weight for foreground (assuming mask imbalance)
    weight_map = target * 2 + 1  # Weight 2 for foreground, 1 for background
    weighted_bce = (weight_map * (target * torch.log(pred + smooth) + (1 - target) * torch.log(1 - pred + smooth))).mean()
    return weight * dice + (1 - weight) * (-weighted_bce)

# ### Model Design: Attention U-Net with Residual Connections
class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionGate, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(F_int)
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(F_int)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

class MultiTaskAttentionUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels_seg=1, num_classes_class=3):
        super(MultiTaskAttentionUNet, self).__init__()

        # Encoder 1
        self.enc1_conv1 = nn.Conv2d(in_channels, 64, 3, padding=1)
        self.enc1_conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.enc1_bn1 = nn.BatchNorm2d(64)
        self.enc1_bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)

        # Encoder 2
        self.enc2_conv1 = nn.Conv2d(64, 128, 3, padding=1)
        self.enc2_conv2 = nn.Conv2d(128, 128, 3, padding=1)
        self.enc2_bn1 = nn.BatchNorm2d(128)
        self.enc2_bn2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2, 2)

        # Encoder 3
        self.enc3_conv1 = nn.Conv2d(128, 256, 3, padding=1)
        self.enc3_conv2 = nn.Conv2d(256, 256, 3, padding=1)
        self.enc3_bn1 = nn.BatchNorm2d(256)
        self.enc3_bn2 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2, 2)

        # Bottleneck
        self.bottleneck_conv1 = nn.Conv2d(256, 512, 3, padding=1)
        self.bottleneck_conv2 = nn.Conv2d(512, 512, 3, padding=1)
        self.bottleneck_bn1 = nn.BatchNorm2d(512)
        self.bottleneck_bn2 = nn.BatchNorm2d(512)
        self.dropout = nn.Dropout(0.3)

        # Decoder 3
        self.upconv3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3_conv1 = nn.Conv2d(512, 256, 3, padding=1)  # 512 because of concatenation
        self.dec3_conv2 = nn.Conv2d(256, 256, 3, padding=1)
        self.dec3_bn1 = nn.BatchNorm2d(256)
        self.dec3_bn2 = nn.BatchNorm2d(256)
        self.att3 = AttentionGate(F_g=256, F_l=256, F_int=128)

        # Decoder 2
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2_conv1 = nn.Conv2d(256, 128, 3, padding=1)  # 256 because of concatenation
        self.dec2_conv2 = nn.Conv2d(128, 128, 3, padding=1)
        self.dec2_bn1 = nn.BatchNorm2d(128)
        self.dec2_bn2 = nn.BatchNorm2d(128)
        self.att2 = AttentionGate(F_g=128, F_l=128, F_int=64)

        # Decoder 1
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1_conv1 = nn.Conv2d(128, 64, 3, padding=1)  # 128 because of concatenation
        self.dec1_conv2 = nn.Conv2d(64, out_channels_seg, 3, padding=1)
        self.dec1_bn1 = nn.BatchNorm2d(64)
        self.att1 = AttentionGate(F_g=64, F_l=64, F_int=32)

        # Classifier
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(512, 256),  # Changed from 2048 to match bottleneck
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes_class)
        )

    def forward(self, x):
        # Encoder 1
        enc1 = F.relu(self.enc1_bn1(self.enc1_conv1(x)))
        enc1 = F.relu(self.enc1_bn2(self.enc1_conv2(enc1)) + enc1)
        pool1 = self.pool1(enc1)

        # Encoder 2
        enc2 = F.relu(self.enc2_bn1(self.enc2_conv1(pool1)))
        enc2 = F.relu(self.enc2_bn2(self.enc2_conv2(enc2)) + enc2)
        pool2 = self.pool2(enc2)

        # Encoder 3
        enc3 = F.relu(self.enc3_bn1(self.enc3_conv1(pool2)))
        enc3 = F.relu(self.enc3_bn2(self.enc3_conv2(enc3)) + enc3)
        pool3 = self.pool3(enc3)

        # Bottleneck
        bottleneck = F.relu(self.bottleneck_bn1(self.bottleneck_conv1(pool3)))
        bottleneck = F.relu(self.bottleneck_bn2(self.bottleneck_conv2(bottleneck)) + bottleneck)
        bottleneck = self.dropout(bottleneck)

        # Decoder 3 with attention
        up3 = self.upconv3(bottleneck)
        att3 = self.att3(g=up3, x=enc3)
        dec3_input = torch.cat([up3, att3], dim=1)
        dec3 = F.relu(self.dec3_bn1(self.dec3_conv1(dec3_input)))
        dec3 = F.relu(self.dec3_bn2(self.dec3_conv2(dec3)) + dec3)

        # Decoder 2 with attention
        up2 = self.upconv2(dec3)
        att2 = self.att2(g=up2, x=enc2)
        dec2_input = torch.cat([up2, att2], dim=1)
        dec2 = F.relu(self.dec2_bn1(self.dec2_conv1(dec2_input)))
        dec2 = F.relu(self.dec2_bn2(self.dec2_conv2(dec2)) + dec2)

        # Decoder 1 with attention
        up1 = self.upconv1(dec2)
        att1 = self.att1(g=up1, x=enc1)
        dec1_input = torch.cat([up1, att1], dim=1)
        dec1 = F.relu(self.dec1_bn1(self.dec1_conv1(dec1_input)))
        seg_out = self.dec1_conv2(dec1)

        # Classification
        cls_out = self.classifier(bottleneck)

        return seg_out, cls_out
# ### Evaluation Metrics: IoU, Dice, Precision, Recall with post-processing
def evaluate(model, loader, threshold=0.5, return_metrics=False):
    model.eval()
    total_loss = 0.0
    total_seg_loss = 0.0
    total_cls_loss = 0.0
    total_dice = 0.0
    total_iou = 0.0
    total_pixel_acc = 0.0
    total_correct = 0
    total_samples = 0
    all_preds = []
    all_labels = []
    smooth = 1e-5
    structure = np.ones((3, 3))  # For morphological closing

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            if batch is None:
                continue
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            labels = batch['label'].to(device)
            seg_pred, cls_pred = model(images)
            loss, seg_loss, cls_loss = combined_loss(seg_pred, masks, cls_pred, labels)
            total_loss += loss.item()
            total_seg_loss += seg_loss
            total_cls_loss += cls_loss

            # Test multiple thresholds for IoU optimization
            best_iou = 0.0
            best_threshold = threshold
            for th in [0.3, 0.4, 0.5, 0.6, 0.7]:
                seg_pred_binary = (torch.sigmoid(seg_pred) > th).float().cpu().numpy()
                for i in range(seg_pred_binary.shape[0]):
                    seg_pred_binary[i, 0] = binary_closing(seg_pred_binary[i, 0], structure=structure).astype(np.float32)
                seg_pred_binary = torch.tensor(seg_pred_binary).to(device)

                intersection = (seg_pred_binary * masks).sum()
                union = seg_pred_binary.sum() + masks.sum() - intersection
                iou = (intersection + smooth) / (union + smooth)
                if iou.item() > best_iou:
                    best_iou = iou.item()
                    best_threshold = th
            seg_pred_binary = (torch.sigmoid(seg_pred) > best_threshold).float().cpu().numpy()
            for i in range(seg_pred_binary.shape[0]):
                seg_pred_binary[i, 0] = binary_closing(seg_pred_binary[i, 0], structure=structure).astype(np.float32)
            seg_pred_binary = torch.tensor(seg_pred_binary).to(device)

            intersection = (seg_pred_binary * masks).sum()
            union = seg_pred_binary.sum() + masks.sum() - intersection
            iou = (intersection + smooth) / (union + smooth)
            total_iou += iou.item()

            dice = (2. * intersection + smooth) / (seg_pred_binary.sum() + masks.sum() + smooth)
            total_dice += dice.item()

            pixel_acc = (seg_pred_binary == masks).float().mean()
            total_pixel_acc += pixel_acc.item()

            cls_pred_labels = cls_pred.argmax(dim=1)
            total_correct += (cls_pred_labels == labels).sum().item()
            total_samples += labels.size(0)
            all_preds.extend(cls_pred_labels.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(loader)
    avg_seg = total_seg_loss / len(loader)
    avg_cls = total_cls_loss / len(loader)
    avg_dice = total_dice / len(loader)
    avg_iou = total_iou / len(loader)
    avg_pixel_acc = total_pixel_acc / len(loader)
    avg_acc = total_correct / total_samples
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=0)

    if return_metrics:
        print(f"Best Threshold: {best_threshold}, IoU: {avg_iou:.4f}")
        return avg_loss, avg_seg, avg_cls, avg_dice, avg_iou, avg_pixel_acc, avg_acc, precision, recall, f1
    return avg_loss

# ### Training Strategy: Optimization, Learning Rate Scheduling
def combined_loss(seg_pred, seg_true, cls_pred, cls_true, seg_weight=0.7, cls_weight=0.3):
    seg_loss = dice_bce_loss(seg_pred, seg_true)
    cls_loss = F.cross_entropy(cls_pred, cls_true)
    return seg_weight * seg_loss + cls_weight * cls_loss, seg_loss.item(), cls_loss.item()

def train_model(model, train_loader, val_loader, optimizer, scheduler, epochs=5, threshold=0.5):
    best_iou = 0.0
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        total_seg_loss = 0.0
        total_cls_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            if batch is None:
                continue
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            labels = batch['label'].to(device)
            optimizer.zero_grad()
            seg_pred, cls_pred = model(images)
            loss, seg_loss, cls_loss = combined_loss(seg_pred, masks, cls_pred, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            total_seg_loss += seg_loss
            total_cls_loss += cls_loss

        # Validation
        model.eval()
        val_loss, _, _, _, val_iou, _, _, _, _, _ = evaluate(model, val_loader, threshold=threshold, return_metrics=True)
        scheduler.step(val_loss)

        if epoch % 5 == 0 or epoch == epochs - 1:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}, Val IoU: {val_iou:.4f}")
            if val_iou > best_iou:
                best_iou = val_iou
                torch.save(model.state_dict(), 'best_model.pth')
                print(f"New best model saved with IoU: {best_iou:.4f}")

dataset = BreastUltrasoundDataset(image_dirs)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

model = MultiTaskAttentionUNet().to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, verbose=True)

train_model(model, train_loader, val_loader, optimizer, scheduler, epochs=5)

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, 'breast_ultrasound_final_model.pth')

print("\nFinal model saved to 'breast_ultrasound_final_model.pth'\n")
print("\nTraining and evaluation complete!")

Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Contents of /content/drive/MyDrive:
['SEO.gdoc', 'Acess Control codes.gdoc', 'archive (14)', 'Colab Notebooks']

Found dataset at: /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT
Contents of dataset folder: ['malignant', 'benign', 'normal']
Found directory: /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/benign
  Contains 437 images and 437 masks
Found directory: /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/malignant
  Contains 210 images and 210 masks
Found directory: /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/normal
  Contains 133 images and 133 masks
Skipping sample with empty mask: /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/normal/normal (1)_mask.png
Skipping sample with empty mask: /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/normal/normal (10)_mask.png
Skipping sam

Epoch 1/5:   0%|          | 0/130 [00:00<?, ?it/s]

Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/benign/benign (102).png: Unexpected type <class 'dict'>
Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/benign/benign (330).png: img should be PIL Image. Got <class 'dict'>
Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/malignant/malignant (148).png: img should be PIL Image. Got <class 'dict'>
Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/benign/benign (337).png: Unexpected type <class 'dict'>


Epoch 1/5:   1%|          | 1/130 [00:00<01:02,  2.06it/s]

Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/benign/benign (210).png: Unexpected type <class 'dict'>
Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/benign/benign (269).png: img should be PIL Image. Got <class 'dict'>
Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/malignant/malignant (156).png: img should be PIL Image. Got <class 'dict'>
Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/benign/benign (367).png: Unexpected type <class 'dict'>


...

Evaluating:  97%|█████████▋| 32/33 [00:11<00:00,  2.32it/s]

Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/malignant/malignant (39).png: Unexpected type <class 'dict'>


Evaluating: 100%|██████████| 33/33 [00:11<00:00,  2.80it/s]

Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/benign/benign (233).png: img should be PIL Image. Got <class 'dict'>
Error loading /content/drive/MyDrive/archive (14)/Dataset_BUSI_with_GT/benign/benign (356).png: img should be PIL Image. Got <class 'dict'>
Best Threshold: 0.3, IoU: 1.0000
Epoch 5/5, Loss: 0.7529, Val IoU: 1.0000



Final model saved to 'breast_ultrasound_final_model.pth'


Training and evaluation complete!
